In [0]:
# Basic imports from pytorch and torchvision library
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


class Net(nn.Module):
    def __init__(self):
        # The network components have to be defined in this method
        super(Net, self).__init__()
        self.conv1_b1 = nn.Conv2d(1, 20, (5,3), 1, stide=, padding=)
        self.conv1_b2 = nn.Conv2d(1, 20, 7, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, myInput1, myINput2):
      # The data flow is to be defined in this method
        x1 = F.relu(self.conv1_b1(myInput1))
        x2 = F.relu(self.conv1_b2(myInput2))
        x = torch.concat(x1, x2, dim=1)
        (batchDim, channel, height, width)
        (64, 20+20, height, width)
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):             # Fetch the data and labels from dataloader
        data, target = data.cuda(), target.cuda()                         # Move data and labels to GPU
        optimizer.zero_grad()
        output = model(data)                                              # Make predictions from the data
        loss = F.nll_loss(output, target) + myRegularizationFunction()    # Calculate loss function
        loss.backward()                                                   # Calculate gradients
        optimizer.step()                                                  # Do a backpropgation step
        # Below code is for printing the progress
        if batch_idx % 10000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:                                  # Fetch the data and labels from dataloader
            data, target = data.to(device), target.to(device)             # Move data and labels to GPU
            output = model(data)                                          # Predict using the model
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()         # Calculate the total number of correct samples

    test_loss /= len(test_loader.dataset)
    # Below code is for printing the progress
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    use_cuda = True

    torch.manual_seed(1)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    # Train data loader : This is basically a sample feeder to the network
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=64, shuffle=True, **kwargs)
    # Test data loader : This is again basically a sample feeder to the network
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=1000, shuffle=True, **kwargs)


    model = Net().cuda()                                                    # Call the network and push it to GPU
    optimizer = optim.SGD([model.parameters(), ], lr=0.01, momentum=0.5)    # Define the optimizer with its parameters and pass the parameters to be trained

    # Do training and testing epochs
    for epoch in range(1, 10 + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

    # Save model parameters
    torch.save(model.state_dict(),"mnist_cnn.pt")
        
if __name__ == '__main__':
    main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300039

Test set: Average loss: 0.1016, Accuracy: 9664/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.145283

Test set: Average loss: 0.0613, Accuracy: 9827/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.054103

Test set: Average loss: 0.0564, Accuracy: 9809/10000 (98%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.019030

Test set: Average loss: 0.0411, Accuracy: 9863/10000 (99%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 0.011021

Test set: Average loss: 0.0386, Accuracy: 9868/10000 (99%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.127844

Test set: Average loss: 0.0339, Accuracy: 9888/10000 (99%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.029382

Test set: Average loss: 0.0339, Accuracy: 9876/10000 (99%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.004214

Test set: Average loss: 0.0398, Accuracy: 9874/10000 (99%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.093632

Test set: Average loss: 0.0290, Accuracy: 9914/10000 (99%)

Train Epoch: 10 [0/60000 (0%)]	Loss: 0.123286
